In [7]:
import os, argparse, datetime, logging, traceback, time
import time, random, warnings
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

def main(IdDaysDict, verbose=True):
    ### Output
    col = 'StockID, StockName, Date, KRaw, DRaw, JRaw, K, D, J'
    dfStockKDJ = pd.DataFrame(columns = col.split(', '))

    ### XpathSetting ###
    # XpathSettingButton ='/html/body/div[23]/input'
    # XpathIdxLengthInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[4]/td[2]/input'
    # XpathShiftInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td[2]/input'
    XpathSettingConfirmButton = '/html/body/div[21]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
    # XpathPreviousDayButton = '/html/body/div[20]/nobr/table/tbody/tr/td[2]/nobr/input[2]'
    XpathAnchor = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[2]/td/canvas'
    XpathStockName = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[1]/td/table/tbody/tr/td[2]/nobr/b'
    XpathDate = '/html/body/div[3]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'
    XpathK = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[1]/font'
    XpathD = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[2]/font'
    XpathJ = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[3]/font'

    ### webdriver
    if verbose: print('Opening Webdirver...')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--start-maximized')
    # chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
    driver.maximize_window()
    FirstSetting = True
    
    for STOCK_ID, DAY in IdDaysDict.items():
        if verbose:print(f'Fetching  StockID  :  {STOCK_ID}  information ...\t(1/{DAY})\t', end = '')
        url = f'https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={STOCK_ID}&CHT_CAT2=DATE'
        driver.get(url)

        settingButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'StockCanvas_btnConfig')))
        settingButton.click()

        if FirstSetting:
            idxLengthInput = driver.find_element(By.ID, 'CFG_InitIndexLength')
            shiftInput = driver.find_element(By.ID, 'CFG_CtrlShiftCnt')
            settingConfirmButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, XpathSettingConfirmButton)))
            idxLengthInput.send_keys('\b\b\b1')
            shiftInput.send_keys('\b\b1')
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))
            settingConfirmButton.click()
        else:
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))

        for i in range(DAY):
            stockName = driver.find_element(By.XPATH, XpathStockName).text.split()[1]
            date = driver.find_element(By.XPATH, XpathDate).text.strip()
            KRaw, DRaw, JRaw = map(lambda x: driver.find_element(By.XPATH, x).text[3:], (XpathK, XpathD, XpathJ))
            K, D, J = map(lambda x: float(x[:-1]), (KRaw, DRaw, JRaw))
            dfStockKDJ.loc[len(dfStockKDJ)] = tuple([STOCK_ID, stockName, date, KRaw, DRaw, JRaw, K, D, J])
            if verbose: print(f'|| {STOCK_ID} | {stockName} | {date} | K: {KRaw:^6s} | D: {DRaw:^6s} | J: {JRaw:^6s} ||')
            if i == DAY - 1: break # early break

            # change date
            if verbose: print(f"Fetching the previous day's information ...\t({i+2}/{DAY})\t", end = '')
            previousDayButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnCtrlShiftLeft')))
            previousDayButton.click()
            time.sleep(1)
        if verbose: print()
    driver.close()
    if verbose: print('All query is done!!')
    return dfStockKDJ

In [8]:
### Example
# Input
IdDaysDict = {'1101': 2,
              '0050': 3}

df = main(IdDaysDict)
display(df)

Opening Webdirver...


C:\Users\Mia\AppData\Local\Temp\ipykernel_1284\1020232282.py:37: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)
C:\Users\Mia\AppData\Local\Temp\ipykernel_1284\1020232282.py:37: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=chrome_options)


Fetching  StockID  :  1101  information ...	(1/2)	|| 1101 | 台泥 | 2022/09/16 (五) | K: 54.4▼  | D: 68.5▼  | J: 26.3▼  ||
Fetching the previous day's information ...	(2/2)	|| 1101 | 台泥 | 2022/09/15 (四) | K: 78.3▼  | D: 75.5▲  | J: 83.8▼  ||

Fetching  StockID  :  0050  information ...	(1/3)	|| 0050 | 元大台灣50 | 2022/09/16 (五) | K: 30.4▼  | D: 32.8▼  | J: 25.5▼  ||
Fetching the previous day's information ...	(2/3)	|| 0050 | 元大台灣50 | 2022/09/15 (四) | K: 40.3▲  | D:  34▲   | J:  53▼   ||
Fetching the previous day's information ...	(3/3)	|| 0050 | 元大台灣50 | 2022/09/14 (三) | K: 40.1▼  | D: 30.8▲  | J: 58.7▼  ||

All query is done!!


,StockID,StockName,Date,KRaw,DRaw,JRaw,K,D,J
0,1101,台泥,2022/09/16 (五),54.4▼,68.5▼,26.3▼,54.4,68.5,26.3
1,1101,台泥,2022/09/15 (四),78.3▼,75.5▲,83.8▼,78.3,75.5,83.8
2,0050,元大台灣50,2022/09/16 (五),30.4▼,32.8▼,25.5▼,30.4,32.8,25.5
3,0050,元大台灣50,2022/09/15 (四),40.3▲,34▲,53▼,40.3,34.0,53.0
4,0050,元大台灣50,2022/09/14 (三),40.1▼,30.8▲,58.7▼,40.1,30.8,58.7
